In [1]:
import mediapipe as mp
import cv2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import json

2023-09-28 11:36:14.998760: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
/Users/jhsu12/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:

selected_columns = pd.read_json('./results2nd/inference_args.json')
def load_relevant_data_subset(pq_path):
    return pd.read_parquet(pq_path, columns=selected_columns['selected_columns'])
    #return pd.read_parquet(pq_path)

def predict(xyz_np):
    interpreter = tf.lite.Interpreter('./results2nd/model.tflite')
    found_signatures = list(interpreter.get_signature_list().keys())
    prediction_fn = interpreter.get_signature_runner("serving_default")

    #pq_file = './output.parquet'
    #xyz_np = load_relevant_data_subset(pq_file)
    xyz_np = xyz_np.loc[:, selected_columns['selected_columns']]
    xyz_np = xyz_np.astype('float32')
    prediction = prediction_fn(inputs=xyz_np.values)

    with open ("./character_to_prediction_index.json", "r") as f:
            character_map = json.load(f)
    rev_character_map = {j:i for i,j in character_map.items()}
    prediction_str = "".join([rev_character_map.get(s, "") for s in np.argmax(prediction['outputs'], axis=1)])

    return prediction_str

mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_holistic = mp.solutions.holistic

def create_frame_landmark_df(results, frame, cols):
    landmarks = pd.DataFrame()
   

    landmarks.loc[0, cols] = np.nan
    landmarks = landmarks.assign(frame=frame)
    
    if results.face_landmarks:
        for i, point in enumerate(results.face_landmarks.landmark):
            landmarks.loc[0, [f'x_face_{i}', f'y_face_{i}', f'z_face_{i}']] = [point.x, point.y, point.z]
    if results.pose_landmarks:
        for i, point in enumerate(results.pose_landmarks.landmark):
            landmarks.loc[0, [f'x_pose_{i}', f'y_pose_{i}', f'z_pose_{i}']] = [point.x, point.y, point.z]
    if results.left_hand_landmarks:
        for i, point in enumerate(results.left_hand_landmarks.landmark):
            landmarks.loc[0, [f'x_left_hand_{i}', f'y_left_hand_{i}', f'z_left_hand_{i}']] = [point.x, point.y, point.z]
    if results.right_hand_landmarks:
        for i, point in enumerate(results.right_hand_landmarks.landmark):
            landmarks.loc[0, [f'x_right_hand_{i}', f'y_right_hand_{i}', f'z_right_hand_{i}']] = [point.x, point.y, point.z]
    
    return landmarks

def do_capture_loop(cols):
    #try:
    # For webcam input:
    landmarks_all = []
    text = ""
    cap = cv2.VideoCapture(0)
    with mp_holistic.Holistic(
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5) as holistic:
        frame = 0
        while cap.isOpened():
            frame += 1
            
            success, image = cap.read()
            if not success:
                print("Ignoring empty camera frame.")
        # If loading a video, use 'break' instead of 'continue'.
                continue

            # To improve performance, optionally mark the image as not writeable to
            # pass by reference.
            image.flags.writeable = False
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            results = holistic.process(image)
            
            # Create landmark df
            landmarks = create_frame_landmark_df(results, frame, cols)
            landmarks_all.append(landmarks)
            
            if frame%4 == 0:
                #landmarks = pd.concat(landmarks_all).reset_index(drop=True).to_parquet('output.parquet')
                landmarks = pd.concat(landmarks_all).reset_index(drop=True)
                # predict
                predict_str = predict(landmarks)
                text = predict_str
                print(f"frame: {frame}, pred: {predict_str}")
                
            
            # Draw landmark annotation on the image.
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
            '''
            mp_drawing.draw_landmarks(
                image,
                results.face_landmarks,
                mp_holistic.FACEMESH_CONTOURS,
                landmark_drawing_spec=None,
                connection_drawing_spec=mp_drawing_styles
                .get_default_face_mesh_contours_style())
            mp_drawing.draw_landmarks(
                image,
                results.pose_landmarks,
                mp_holistic.POSE_CONNECTIONS,
                landmark_drawing_spec=mp_drawing_styles
                .get_default_pose_landmarks_style())
            '''
            
            # Rectangle properties
            rect_x = 0
            rect_y = 500  # Adjust this value to control the position of the rectangle
            rect_width = 400
            rect_height = 200
            rect_color = (0, 0, 0)  # Black color in BGR format


            # Define the text and background colors
            text_color = (255, 255, 255)  # White text in BGR format
            background_color = (0, 0, 0)  # Black background in BGR format

            # Add text to the frame with a black background
            
            font = cv2.FONT_HERSHEY_SIMPLEX
            font_scale = 1
            thickness = 1

            # Calculate text position to center it in the rectangle
            (text_width, text_height), baseline = cv2.getTextSize(text, font, font_scale, thickness)
            text_x = (rect_width - text_width) // 2 + rect_x
            text_y = (rect_height - text_height) // 2 + rect_y + text_height


            # Create a black rectangle as a background for the text
       
            cv2.rectangle(image, (rect_x, rect_y), (rect_x + rect_width, rect_y + rect_height), rect_color, -1)

            # Put the white text on the black background
            frame_with_text = cv2.putText(image, text, (text_x, text_y), font, font_scale, text_color, thickness)
        
            # Flip the image horizontally for a selfie-view display.
            #cv2.imshow('MediaPipe Holistic', cv2.flip(image, 1))

            

            cv2.imshow('MediaPipe Holistic', frame_with_text)
            if cv2.waitKey(5) & 0xFF == 27:
                break
    #except:
    #   return landmarks_all
    cap.release()
    cv2.destroyAllWindows()
    return landmarks_all

if __name__ == "__main__":
    pq_file = './input/train_landmarks/1255240050.parquet'
    xyz = pd.read_parquet(pq_file)
    do_capture_loop(xyz.columns[1:])
    #print(landmarks)
    #landmarks = pd.concat(landmarks).reset_index(drop=True).to_parquet('output.parquet')

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


frame: 4, pred: w
frame: 8, pred: 11
frame: 12, pred: +21-
frame: 16, pred: +22-2
frame: 20, pred: 12 p
frame: 24, pred: 12 w
frame: 28, pred: 1266 
frame: 32, pred: 1226
frame: 36, pred: +226
frame: 40, pred: +216
frame: 44, pred: +226
frame: 48, pred: +226
frame: 52, pred: +226
frame: 56, pred: +226-11
frame: 60, pred: +226-11
frame: 64, pred: +226-11-1
frame: 68, pred: +266-11-1
frame: 72, pred: +26-11-10-1
frame: 76, pred: +26-11-0-1
frame: 80, pred: +26-11-01-1
frame: 84, pred: +26-11-00-1
frame: 88, pred: +26-11-10-1
frame: 92, pred: +26-11-10-1
frame: 96, pred: +26-11-00-1
frame: 100, pred: +26-11-00-1
frame: 104, pred: +26-11-00-1


KeyboardInterrupt: 

: 